Programming Assignment: Градиентный бустинг над решающими деревьями
    
Данное задание основано на материалах лекций по композициям алгоритмов.

Вы научитесь:

работать с градиентным бустингом и подбирать его гиперпараметры
сравнивать разные способы построения композиций
понимать, в каком случае лучше использовать случайный лес, а в каком — градиентный бустинг
использовать метрику log-loss

In [87]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

# 1. Загрузите выборку из файла gbm-data.csv с помощью pandas и преобразуйте ее в массив numpy 
# В первой колонке файла с данными записано, была или нет реакция. 
# Все остальные колонки (d1 - d1776) содержат различные характеристики молекулы, такие как размер, форма и т.д. 
# Разбейте выборку на обучающую и тестовую, используя функцию train_test_split с параметрами test_size = 0.8 и random_state = 241.
df = pd.read_csv('gbm-data.csv')
X = df.loc[:, 'D1':].values
y = df['Activity'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=241)

# 2. Обучите GradientBoostingClassifier с параметрами n_estimators=250, verbose=True, random_state=241 и 
# для каждого значения learning_rate из списка [1, 0.5, 0.3, 0.2, 0.1] проделайте следующее:
# - Используйте метод staged_decision_function для предсказания качества на обучающей и тестовой выборке на каждой итерации.
# - Преобразуйте полученное предсказание с помощью сигмоидной функции по формуле 1 / (1 + e^{−y_pred}), где y_pred — предсказанное значение.
# - Вычислите и постройте график значений log-loss (которую можно посчитать с помощью функции ) на обучающей и тестовой выборках, а также найдите минимальное значение метрики и номер итерации, на которой оно достигается.

def sigmoid(y_pred):
    return 1.0 / (1.0 + np.exp(-y_pred))


def log_loss_results(X, y, model):
    results = []
    for pred in model.staged_decision_function(X):
        results.append(log_loss(y, [sigmoid(y_pred) for y_pred in pred]))

    return results


def plot_log_loss(learning_rate, log_loss_train, log_loss_test):
    plt.figure()
    plt.plot(log_loss_test, 'r', linewidth=2)
    plt.plot(log_loss_train, 'g', linewidth=2)
    plt.legend(['test', 'train'])
    plt.savefig('plots/rate_' + str(learning_rate) + '.png')
    
    min_log_loss_value = min(log_loss_test)
    min_log_los_index = log_loss_test.index(min_log_loss_value)
    
    return min_log_loss_value, min_log_los_index

def model_test(learning_rate):
    grd = GradientBoostingClassifier(learning_rate=learning_rate, n_estimators=250, verbose=False, random_state=241)
    grd.fit(X_train, y_train)

    log_loss_train = log_loss_results(X_train, y_train, grd)
    log_loss_test = log_loss_results(X_test, y_test, grd)

    return plot_log_loss(learning_rate, log_loss_train, log_loss_test)

min_log_loss_results = {}
for learning_rate in [1, 0.5, 0.3, 0.2, 0.1]:
    min_log_loss_results[learning_rate] = model_test(learning_rate)

# 3. Как можно охарактеризовать график качества на тестовой выборке, начиная с некоторой итерации: 
# переобучение (overfitting) или недообучение (underfitting)? В ответе укажите одно из слов overfitting либо underfitting.

# 4. Приведите минимальное значение log-loss на тестовой выборке и номер итерации, на котором оно достигается, при learning_rate = 0.2.
print("Minimal log-loss value and its index with learning_rate 0.2 = { %.2f %d }" % (min_log_loss_results.get(0.2)[0], min_log_loss_results.get(0.2)[1]))

# 5. На этих же данных обучите RandomForestClassifier с количеством деревьев, равным количеству итераций, 
# на котором достигается наилучшее качество у градиентного бустинга из предыдущего пункта, 
# c random_state=241 и остальными параметрами по умолчанию. 
# Какое значение log-loss на тесте получается у этого случайного леса? 
# (Не забывайте, что предсказания нужно получать с помощью функции predict_proba. 
# В данном случае брать сигмоиду от оценки вероятности класса не нужно)

/usr/lib64/python2.7/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Minimal log-loss value and its index with learning_rate 0.2 = { 0.53 36 }


In [83]:
clf = RandomForestClassifier(n_estimators=36, random_state=241)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_test)[:, 1]
test_log_loss = log_loss(y_test, y_pred)
print("Answer 3 is %.2f" % test_log_loss)

Answer 3 is 0.54
